In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/Colab Notebooks/dataset.zip"
extract_path = "/content/drive/MyDrive/Colab Notebooks/Ewaste1ws"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Extracted to {extract_path}")

import zipfile
import os

zip_path = "/content/drive/MyDrive/Colab Notebooks/Ewaste roboflow.zip"
extract_path = "/content/drive/MyDrive/Colab Notebooks/Ewaste2"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Extracted to {extract_path}")

In [ ]:
import os
import shutil
import random

def create_splits(dataset_path, output_path, train_ratio=0.7, val_ratio=0.2):
    """
    Split a dataset into train, test, and validate folders.

    Args:
        dataset_path: Path to the original dataset (Ewaste1).
        output_path: Path to save the split dataset.
        train_ratio: Proportion of data for training.
        val_ratio: Proportion of data for validation.
    """
    os.makedirs(output_path, exist_ok=True)
    splits = ['train', 'test', 'validate']
    for split in splits:
        os.makedirs(os.path.join(output_path, split, 'images'), exist_ok=True)

    # Iterate through each class folder
    for class_name in os.listdir(dataset_path):
        class_dir = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_dir):
            files = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
            random.shuffle(files)

            train_split = int(len(files) * train_ratio)
            val_split = int(len(files) * (train_ratio + val_ratio))

            for i, file_name in enumerate(files):
                src_path = os.path.join(class_dir, file_name)
                if i < train_split:
                    dest_path = os.path.join(output_path, 'train', 'images', file_name)
                elif i < val_split:
                    dest_path = os.path.join(output_path, 'validate', 'images', file_name)
                else:
                    dest_path = os.path.join(output_path, 'test', 'images', file_name)

                shutil.copy(src_path, dest_path)

# Paths
ewaste1_path = '/content/drive/MyDrive/Colab Notebooks/Ewaste1ws'
output_path = '/content/drive/MyDrive/Colab Notebooks/Ewaste1'

# Create splits
create_splits(ewaste1_path, output_path)
print("Splits created successfully!")

In [ ]:
import os
import shutil

def combine_datasets(ewaste1_path, roboflow_path, output_path):
    """Combines Ewaste1 (image classification structure) with Roboflow (YOLO) dataset."""

    os.makedirs(output_path, exist_ok=True)
    os.makedirs(os.path.join(output_path, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_path, "labels"), exist_ok=True)

    img_count = 0


    print("Processing Ewaste1...")
    for class_name in os.listdir(ewaste1_path):
        class_dir = os.path.join(ewaste1_path, class_name)
        if os.path.isdir(class_dir):
            for img_file in os.listdir(class_dir):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_dir, img_file)
                    new_img_name = f"ewaste1_{img_count}.jpg"
                    new_img_path = os.path.join(output_path, "images", new_img_name)
                    shutil.copy(img_path, new_img_path)


                    label_content = f"{class_number_map[class_name]} 0.5 0.5 1 1\n"
                    label_filename = f"ewaste1_{img_count}.txt"
                    label_path = os.path.join(output_path, "labels", label_filename)
                    with open(label_path, "w") as f:
                        f.write(label_content)

                    img_count += 1


    print("Processing Roboflow dataset...")
    for split in ["train", "valid", "test"]:
        split_path = os.path.join(roboflow_path, split)
        if os.path.isdir(split_path):
            img_dir = os.path.join(split_path, "images")
            label_dir = os.path.join(split_path, "labels")

            if os.path.exists(img_dir) and os.path.exists(label_dir):

                for filename in os.listdir(img_dir):
                    img_path = os.path.join(img_dir, filename)
                    new_img_name = f"roboflow_{img_count}_{filename}"
                    new_img_path = os.path.join(output_path, "images", new_img_name)
                    shutil.copy(img_path, new_img_path)


                    label_file = filename.replace(os.path.splitext(filename)[1], ".txt")
                    label_path = os.path.join(label_dir, label_file)
                    if os.path.exists(label_path):
                        new_label_path = os.path.join(output_path, "labels", f"roboflow_{img_count}_{label_file}")
                        shutil.copy(label_path, new_label_path)
                    img_count += 1

    print("Combination Complete!")

def ensure_directory_exists(path):
    """Creates a directory if it doesn't exist."""
    if not os.path.exists(path):
        os.makedirs(path)


def create_class_number_map(path):
    """Create a map of class names to class indices (integers)"""

    class_names = sorted(os.listdir(path))
    class_number_map = {class_name: i for i, class_name in enumerate(class_names)}
    return class_number_map


ewaste1_path = '/content/Ewaste1'
roboflow_path = '/content/Ewaste2'
output_path = '/content/combined_dataset'


os.makedirs(output_path, exist_ok = True)


class_number_map = create_class_number_map(ewaste1_path)
print(class_number_map)

ensure_directory_exists(ewaste1_path)

combine_datasets(ewaste1_path, roboflow_path, output_path)

In [ ]:
!pip install torch torchvision ultralytics matplotlib

import os
import torch

repo_path = "/content/drive/MyDrive/ColabNotebooks/yolov5"
if not os.path.exists(repo_path):
    !git clone https://github.com/ultralytics/yolov5 {repo_path}

%cd {repo_path}

!git pull

!pip install -r requirements.txt

if torch.cuda.is_available():
    device = "cuda"
    print("CUDA is available. Training on GPU.")
else:
    device = "cpu"
    print("CUDA is not available. Training on CPU.")

dataset_path = "/content/drive/MyDrive/ColabNotebooks/combined_dataset/dataset.yaml"
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset YAML file not found at {dataset_path}")

!python train.py \
    --img 640 \
    --batch 16 \
    --epochs 10 \
    --data {dataset_path} \
    --weights yolov5s.pt \
    --device {device}

best_pt_path = f"{repo_path}/runs/train/exp/weights/best.pt"

if os.path.exists(best_pt_path):
    print(f"'best.pt' found at {best_pt_path}")

    drive_save_path = "/content/drive/MyDrive/trained_model.pt"
    !cp {best_pt_path} {drive_save_path}
    print(f"Model weights saved to {drive_save_path}")
else:
    print("Model weights could not be saved because 'best.pt' does not exist. Check training logs for errors.")

In [ ]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/ColabNotebooks/yolov51/yolov5s.pt', force_reload=True)

img_path = '/content/drive/MyDrive/ColabNotebooks/download.jpeg'

results = model(img_path)

results.print()

results.show()